# Analyze prototype elements

### Count GUI elements

In [1]:
import os
import re
import csv
from bs4 import BeautifulSoup

generated_guis_path = '../generated_guis'
app_ids = ['12740', '14283', '18782', '20947', '22151', '27360', '27382', '27707', '30982', '31390', '32310', '3261', '33383', '34346', '34517', '34527', '35526', '3727', '37505', '38961', '40673', '43872', '43977', '44756', '47926', '49794', '53054', '53469', '54377', '54468', '56905', '58124', '59429', '59576', '61851', '63575', '64858', '65592', '67044', '68368', '69574', '69587', '70410', '8640']
method_names = ['instruction', 'pd_zs', 'pd_fs', 'ref_instruction']
dimension_target = (375, 667)

In [2]:
def get_html_files():
    files = []
    for app_id in app_ids:
        for method in method_names:
            html_path = os.path.join(generated_guis_path, str(app_id), f'{method}.html')
            if os.path.isfile(html_path):
                files.append({'UI_Number': app_id, 'Method': method, 'path': html_path})
    return files

def analyze_html(fileinfo):
    with open(fileinfo['path'], encoding='utf-8') as f:
        soup = BeautifulSoup(f, 'html.parser')

    # All buttons (button, input[type=button|submit|reset])
    buttons = soup.find_all('button')
    input_buttons = soup.find_all('input', {'type': re.compile('button|submit|reset', re.I)})
    all_buttons = buttons + input_buttons
    diff_buttons = set()

    for b in all_buttons:
        key = (b.get('id'), b.get('name'), b.get_text(strip=True), b.get('value'))
        diff_buttons.add(key)

    # Clickable buttons (with onclick or type submit/reset/button)
    clickable_buttons = [b for b in all_buttons if b.has_attr('onclick') or b.get('type') in ['submit', 'button', 'reset']]

    # Action buttons (with event: onclick, onsubmit, or inside <form>)
    action_buttons = []
    for b in all_buttons:
        if b.has_attr('onclick') or b.has_attr('onsubmit'):
            action_buttons.append(b)
        # Button inside form (submit action)
        parent = b.find_parent('form')
        if parent:
            action_buttons.append(b)

    # All input fields
    input_fields = soup.find_all('input')
    textareas = soup.find_all('textarea')
    selects = soup.find_all('select')
    all_inputs = input_fields + textareas + selects
    diff_inputs = set()
    for inp in all_inputs:
        key = (inp.get('id'), inp.get('name'), inp.get('type'))
        diff_inputs.add(key)

    # Dimension – check for exact dimension in body, html, main container, or iframe
    dimension_found = False
    for tag in [soup.body, soup.html]:
        if tag:
            w = tag.get('width') or tag.get('style')
            h = tag.get('height') or tag.get('style')
            # Typically style="width:375px; height:667px;"
            if w and h:
                if ('375' in str(w) and '667' in str(h)) or ('375' in str(h) and '667' in str(w)):
                    dimension_found = True
    # Occasionally set in div with id/root/container
    for div in soup.find_all('div'):
        style = div.get('style')
        if style and '375px' in style and '667px' in style:
            dimension_found = True

    return {
        'UI_Number': fileinfo['UI_Number'],
        'Method': fileinfo['Method'],
        'All_inputs': len(all_inputs),
        'Diff_inputs': len(diff_inputs),
        'All_buttons': len(all_buttons),
        'Diff_buttons': len(diff_buttons),
        'Clickable_buttons': len(clickable_buttons),
        'Action_buttons': len(set(action_buttons)),
        'Dimension_375x667': 'YES' if dimension_found else 'NO'
    }

In [3]:
html_files = get_html_files()
results = []
for f in html_files:
    res = analyze_html(f)
    results.append(res)

with open('gui_prototype_analysis.csv', 'w', newline='', encoding='utf-8') as csvfile:
    fieldnames = ['UI_Number', 'Method', 'All_inputs', 'Diff_inputs',
                  'All_buttons', 'Diff_buttons', 'Clickable_buttons', 'Action_buttons', 'Dimension_375x667']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames, delimiter=';')
    writer.writeheader()
    for r in results:
        writer.writerow(r)

print("Analysis complete. Results saved to gui_prototype_analysis.csv")

Analysis complete. Results saved to gui_prototype_analysis.csv


In [36]:
visual_df = pd.read_csv('./design_evaluation/gui_visual_analysis.csv', delimiter=';')
visual_df

,UI_Number,Method,Bounds,Nonresponsive,Hidden,Dimensions
0,12740,instruction,0,0,0,NO
1,12740,pd_zs,0,0,0,NO
2,12740,pd_fs,1,1,0,NO
3,12740,ref_instruction,0,1,0,NO
4,14283,instruction,0,1,0,NO
...,...,...,...,...,...,...
171,70410,ref_instruction,0,8,0,NO
172,8640,instruction,0,0,0,NO
173,8640,pd_zs,0,0,0,NO
174,8640,pd_fs,0,1,0,NO


In [31]:
import pandas as pd

all_df = pd.DataFrame(results)
visual_df = pd.read_csv('./design_evaluation/gui_visual_analysis.csv', delimiter=';')

visual_df['Dimensions'] = visual_df['Dimensions'].map({'YES': 1, 'NO': 0})
agg_df = visual_df.groupby(['UI_Number'], as_index=False).sum()

visual_df['all_elements'] = all_df['All_inputs']+all_df['All_buttons']
visual_df['nonresponsive_pct'] = visual_df['Nonresponsive'] / (all_df['All_inputs']+all_df['All_buttons'])
visual_df['covered_pct'] = visual_df['Hidden'] / (all_df['All_inputs']+all_df['All_buttons'])
visual_df['out_of_bounds_pct'] = visual_df['Bounds'] / (all_df['All_inputs']+all_df['All_buttons'])

sorted_df = visual_df.sort_values(['nonresponsive_pct', 'covered_pct', 'out_of_bounds_pct'], ascending=False)

incorrect_dim_guis = [prototype[0] for prototype in visual_df if prototype[5] == 'YES']

# print("GUIs with incorrect dimensions (not 375x667):", incorrect_dim_guis)

agg_df = agg_df.drop('Method', axis=1)
agg_df

visual_df

,UI_Number,Method,Bounds,Nonresponsive,Hidden,Dimensions,all_elements,nonresponsive_pct,covered_pct,out_of_bounds_pct
0,12740,instruction,0,0,0,0,13,0.000000,0.000000,0.000000
1,12740,pd_zs,0,0,0,0,13,0.000000,0.000000,0.000000
2,12740,pd_fs,1,1,0,0,12,0.083333,0.000000,0.083333
3,12740,ref_instruction,0,1,0,0,13,0.076923,0.000000,0.000000
4,14283,instruction,0,1,0,0,2,0.500000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...
171,70410,ref_instruction,0,8,0,0,8,1.000000,0.000000,0.000000
172,8640,instruction,0,0,0,0,4,0.000000,0.000000,0.000000
173,8640,pd_zs,0,0,0,0,3,0.000000,0.000000,0.000000
174,8640,pd_fs,0,1,0,0,9,0.111111,0.000000,0.000000


### Sortiranje GUI glede na neodzivnost, prekritost elementov in prikaz izven dimenzij

In [21]:
desc_df = pd.read_csv('./description_validation/rezultati.csv', delimiter=';')

agg_df = desc_df.groupby(['UI_Number', 'All'], as_index=False)['Correct'].mean()
agg_df['match_pct'] = agg_df['Correct']/agg_df['All']
sorted_df = agg_df.sort_values(['match_pct'], ascending=True)
sorted_df

,UI_Number,All,Correct,match_pct
30,54468,5,2.50,0.500000
15,34346,8,4.25,0.531250
29,54377,10,5.50,0.550000
38,65592,8,6.25,0.781250
31,56905,7,5.50,0.785714
32,58124,6,4.75,0.791667
0,3261,7,5.75,0.821429
26,49794,6,5.00,0.833333
11,30982,9,7.50,0.833333
43,70410,9,7.50,0.833333


In [35]:
list = [3261,3727,8640,12740,18782,20947,22151,27707,30982,31390,32310,33383,34517,34527, 38961,43977,44756,47926,49794,58124,59429,59576,64858,69574,69587]
len(list)*4

100

### Dodatne metrike za ocenjevanje GUI

1. Alignment (Poravnava)

Koliko so elementi na strani poravnani vzdolž vertikalne osi. Manjša standardna devijacija položajev elementov pomeni boljšo poravnavo.

2. Spacing (Razmik med elementi)

Standardna devijacija razmikov med vrhnjimi robovi elementov (vertikalni razmik). Stabilen, enakomeren razmik daje občutek urejenosti, olajša vizualno skeniranje in navigacijo po strani.

3. Overlap Score (Prekrivanje elementov)

Kolikšen delež površine elementov se prekriva. Višje prekrivanje pomeni nižjo uporabniško kakovost (score = 1 − overlap_ratio).

4. Saliency Score (Pomembnost vizualnih elementov)

Koliko pozornosti privlačijo posamezni elementi na sliki, temelji na algoritmu vizualne izrazitosti.
Elementi, ki so bolj izraziti, bolj pritegnejo uporabnikovo pozornost.

5. Color Harmony (Barvna harmonija)

Kombinacija razdalj med barvami (Lab prostor), entropije in kontrasta svetlosti. Višja vrednost pomeni bolj skladno barvno paleto.

In [70]:
import os
import numpy as np
import pandas as pd
import cv2
from pathlib import Path
from itertools import combinations
from colorspacious import cspace_convert
from scipy.spatial.distance import euclidean
from scipy.stats import entropy
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
from playwright.sync_api import sync_playwright
from multiprocessing import Process, Queue

ROOT = Path("../generated_guis")
PROMPTS = ["instruction", "pd_zs", "pd_fs", "ref_instruction"]
VIEWPORT = {"width": 667, "height": 375}
OUT_CSV = "all_ui_metrics_md.csv"
PLOTS_DIR = Path("plots_md")
SCREENSHOTS_DIR = Path("screenshots_md")
for d in [PLOTS_DIR, SCREENSHOTS_DIR]:
   d.mkdir(exist_ok=True)

In [71]:
def parse_rgb(css_color):
    if not css_color or 'transparent' in css_color:
        return None
    try:
        nums = css_color.replace('rgba(','').replace('rgb(','').replace(')','').split(',')
        nums = [int(float(x.strip())) for x in nums[:3]]
        return np.array(nums, dtype=np.uint8)
    except Exception:
        return None

def spacing_score_md(boxes):
    """Spacing skladno z 8dp MD grid"""
    if len(boxes) < 2: return np.nan
    ys_sorted = np.sort([b['top'] for b in boxes])
    diffs = np.diff(ys_sorted)
    compliant = sum(1 for d in diffs if d % 8 == 0)
    score = compliant / len(diffs)
    return float(score)

def overlap_md(boxes):
    """Prekrivanje samo med elementi na isti ploskvi (z-index = 0)"""
    overlaps = 0
    same_plane = [b for b in boxes if int(b.get('z', 0)) == 0]
    for a, b in combinations(same_plane, 2):
        ax1, ay1, ax2, ay2 = a["left"], a["top"], a["left"]+a["width"], a["top"]+a["height"]
        bx1, by1, bx2, by2 = b["left"], b["top"], b["left"]+b["width"], b["top"]+b["height"]
        inter_w = max(0, min(ax2,bx2)-max(ax1,bx1))
        inter_h = max(0, min(ay2,by2)-max(ay1,by2))
        overlaps += inter_w * inter_h
    canvas_area = VIEWPORT['width']*VIEWPORT['height']
    return 1 - min(overlaps / canvas_area, 1.0)

def saliency_ratio(img_path, boxes):
    """Razmerje salience primarnega elementa proti ostalim"""
    sal_map = cv2.saliency.StaticSaliencySpectralResidual_create()
    img = cv2.imread(img_path)
    success, salMap = sal_map.computeSaliency(img)
    if not success: return np.nan
    salMap = (salMap*255).astype("uint8")
    vals = []
    for b in boxes:
        x0, y0 = b['left'], b['top']
        x1, y1 = x0 + b['width'], y0 + b['height']
        crop = salMap[y0:y1, x0:x1]
        if crop.size>0:
            vals.append(float(np.mean(crop))/255.0)
    if not vals: return np.nan
    vals = np.array(vals)
    primary_sal = vals.max()
    ratio = primary_sal / (vals.mean() + 1e-6)
    return float(ratio)

def color_contrast_score(boxes):
    """Kontrast primarne barve proti ostalim"""
    rgbs = [parse_rgb(b.get("bg")) for b in boxes if parse_rgb(b.get("bg")) is not None]
    if len(rgbs) < 2: return np.nan
    rgbs = np.array(rgbs)/255.0
    labs = cspace_convert(rgbs, "sRGB1", "CIELab")
    primary_idx = np.argmax([b.get("width",0)*b.get("height",0) for b in boxes])
    primary_lab = labs[primary_idx]
    other_labs = np.delete(labs, primary_idx, axis=0)
    dists = [euclidean(primary_lab, o) for o in other_labs]
    score = np.mean(dists) / 100.0
    return float(np.clip(score, 0, 1))

def focus_accessibility_metrics_split(page):
    """Focusable ratio & labelled ratio"""
    res = page.evaluate("""
    () => {
        const buttons = Array.from(document.querySelectorAll('button, [role=button]'));
        const total = buttons.length;
        const focusable = buttons.filter(b => b.tabIndex >= 0).length;
        const labelled = buttons.filter(b => b.getAttribute('aria-label') || b.getAttribute('contentDescription')).length;
        return {total, focusable, labelled};
    }
    """)
    if res['total']==0:
        return {"focusable_ratio": np.nan, "labelled_ratio": np.nan}
    return {
        "focusable_ratio": float(res['focusable']/res['total']),
        "labelled_ratio": float(res['labelled']/res['total'])
    }


def get_boxes(page):
    return page.evaluate("""
        () => Array.from(document.querySelectorAll('*'))
            .filter(e => {
                const r = e.getBoundingClientRect();
                return r.width>0 && r.height>0 && getComputedStyle(e).visibility !== 'hidden' && getComputedStyle(e).display !== 'none';
            })
            .map(e => {
                const r = e.getBoundingClientRect();
                const cs = getComputedStyle(e);
                return {
                    tag: e.tagName.toLowerCase(),
                    left: Math.round(r.left),
                    top: Math.round(r.top),
                    width: Math.round(r.width),
                    height: Math.round(r.height),
                    color: cs.color,
                    bg: cs.backgroundColor,
                    z: (cs.zIndex || '0'),
                    pointer: cs.pointerEvents
                };
            })
    """)

def evaluate_gui_md(page, shot_path):
    """Integracija vseh Material Design aware metrik za eno stran"""
    boxes = get_boxes(page)
    spacing_score = spacing_score_md(boxes)
    overlap_score = overlap_md(boxes)
    sal_ratio = saliency_ratio(str(shot_path), boxes)
    color_contrast = color_contrast_score(boxes)
    focus_metrics = focus_accessibility_metrics_split(page)
    metrics = {
        "n_elements": len(boxes),
        "spacing_md": spacing_score,
        "overlap_md": overlap_score,
        "saliency_ratio": sal_ratio,
        "color_contrast": color_contrast,
        **focus_metrics
    }
    return metrics

In [72]:
def run_playwright_md(q: Queue):
    rows = []
    with sync_playwright() as p:
        browser = p.chromium.launch(headless=True)
        context = browser.new_context(viewport=VIEWPORT)
        page = context.new_page()

        appdirs = [d for d in ROOT.iterdir() if d.is_dir()]
        for appdir in tqdm(appdirs, desc="Evaluating GUIs"):
            app_id = appdir.name
            for prompt in PROMPTS:
                html_path = appdir / f"{prompt}.html"
                if not html_path.exists(): continue

                page.goto("file:///" + str(html_path.resolve()))
                page.wait_for_timeout(300)
                shot_path = SCREENSHOTS_DIR / f"{app_id}_{prompt}.png"
                page.screenshot(path=str(shot_path), full_page=True)

                metrics = evaluate_gui_md(page, shot_path)
                metrics.update({"app_id": app_id, "prompt": prompt})
                rows.append(metrics)

        browser.close()
    q.put(rows)

def main_jupyter_safe_md():
    q = Queue()
    p = Process(target=run_playwright_md, args=(q,))
    p.start()
    p.join()
    rows = q.get()

    df = pd.DataFrame(rows)
    df.to_csv(OUT_CSV, index=False)
    print("✅ Rezultati shranjeni v:", OUT_CSV)

    metrics = ["spacing_md","overlap_md","saliency_ratio","color_contrast","focusable_ratio","labelled_ratio"]
    for m in metrics:
        if m in df.columns:
            plt.figure(figsize=(6,4))
            plt.hist(df[m].dropna(), bins=20)
            plt.title(f"Distribucija metrika: {m}")
            plt.xlabel("Vrednost")
            plt.ylabel("Število primerov")
            plt.tight_layout()
            plt.savefig(PLOTS_DIR / f"dist_{m}.png")
            plt.close()
    print("📈 Histogrami shranjeni v:", PLOTS_DIR)

In [ ]:
main_jupyter_safe_md()